**Reference Link:** [RAG Systems Essentials (Analytics Vidhya)](https://courses.analyticsvidhya.com/courses/take/rag-systems-essentials/lessons/60148017-hands-on-deep-dive-into-rag-evaluation-metrics-generator-metrics-i)

# Basic RAG System with LangChain




### Load Dependencies

In [0]:
!pip install langchain==0.3.10
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

In [0]:
!pip install langchain-chroma==0.1.4

## Enter API Tokens

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter your OpenAI Key: ')

## Setup Environment Variables

In [0]:
# import os

# os.environ['OPENAI_API_KEY'] = OPENAI_KEY

### Load Wikipedia Data

In [0]:
import gzip
import json
import requests
from tqdm import tqdm
import sys
import os

# Download a file from a URL
def http_get(url, path) -> None:
    """
    Downloads a URL to a given path on disc
    """
    if os.path.dirname(path) != "":
        os.makedirs(os.path.dirname(path), exist_ok=True)

    req = requests.get(url, stream=True)
    if req.status_code != 200:
        print("Exception when trying to download {}. Response {}".format(url, req.status_code), file=sys.stderr)
        req.raise_for_status()
        return

    download_filepath = path + "_part"
    with open(download_filepath, "wb") as file_binary:
        content_length = req.headers.get("Content-Length")
        total = int(content_length) if content_length is not None else None
        progress = tqdm(unit="B", total=total, unit_scale=True)
        for chunk in req.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive new chunks
                progress.update(len(chunk))
                file_binary.write(chunk)

    os.rename(download_filepath, path)
    progress.close()


wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

In [0]:
import gzip
import json

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

passages = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        #Only add the first paragraph
        passages.append(data['paragraphs'][0])

print("Passages:", len(passages))

In [0]:
# We subset our data so we only use a subset of wikipedia to run things faster
passages = [passage for passage in passages for x in ['fish', 'india', 'cheetah']
              if x in passage.lower().split()]

In [0]:
len(passages)

In [0]:
passages[0]

### Load Open AI LLMs

In [0]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

### Generate LLM Embeddings and store them in Chroma Vector DB

**Chroma Vector DB** is a versatile, open-source vector database designed for managing and querying vector embeddings. It is easy to set up and integrates well with various AI tools and algorithms. Chroma is particularly useful for applications that require rapid and precise retrieval of content represented as embeddings—efficient data formats for text, images, and soon, audio and video.

**Key Features:**
- **Integration with AI Tools:** Chroma supports embedding functions from leading providers like OpenAI, Google, and Hugging Face, allowing for flexible and powerful data handling.
- **Ease of Use:** The database provides default embedding functions, or users can integrate external APIs to generate embeddings.
- **Efficient Querying:** Users can create collections to store embeddings, documents, and metadata. These can be queried to retrieve the most similar items, making information retrieval quick and effective.
- **Flexible API:** Chroma offers a straightforward API that supports both standard operations and custom embedding functions.

For more detailed information, visit the official Chroma documentation [here](https://docs.trychroma.com).


In [0]:
passages[:3]

In [0]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [0]:
# The vectorstore we'll be using
from langchain_chroma import Chroma

# The splitting and chunking strategy
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [0]:
from langchain.docstore.document import Document

docs = [Document(page_content=doc) for doc in passages]

In [0]:
docs[:3]

In [0]:
splitter = RecursiveCharacterTextSplitter(chunk_size=3000,
                                          chunk_overlap=200)
chunked_docs = splitter.split_documents(docs)

In [0]:
chunked_docs[:3]

## Create Vector DB and Retriever

If you have already created `wiki_db`in the previous hands-on session then just load the DB and DO NOT run the following code to create the database again, ignore this when running on Colab

In [0]:
# create vector DB of docs and embeddings - takes 1 min on Colab
chroma_db = Chroma.from_documents(documents=chunked_docs, collection_name='wiki_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./wiki_db")

## Load Vector DB from disk

Run the following code if your vector DB already exists on disk from the previous hands-on session

In [0]:
# load from disk
chroma_db = Chroma(persist_directory="./wiki_db",
                   collection_name='wiki_db',
                   embedding_function=openai_embed_model)

In [0]:
chroma_db

In [0]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                              search_kwargs={"k": 5, "score_threshold": 0.2})

### Build a QA RAG Chain

In [0]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If the answer is not present in the context, just say that you don't know.
            Keep the answer to the point.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

In [0]:
prompt_template.pretty_print()

## RAG Chain - Using LCEL

In [0]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
      |
    prompt_template
      |
    chatgpt
)

In [0]:
query = "What is the fastest animal?"
result = qa_rag_chain.invoke(query)
print(result.content)

In [0]:
query = "Who was the winner of the champions league in 2020?"
result = qa_rag_chain.invoke(query)
print(result.content)

In [0]:
query = "What is the capital of India?"
result = qa_rag_chain.invoke(query)
print(result.content)

In [0]:
query = "Tell me more about it in detail"
result = qa_rag_chain.invoke(query)
print(result.content)

# Conversational RAG System with LangChain

In many Q&A applications, the ability to engage in back-and-forth conversations with users is crucial. This necessitates the application having a form of "memory" to recall past interactions and apply this context to current queries.

This guide focuses on integrating historical messages into the application's logic. Additional details on managing chat history can be found [here](https://python.langchain.com/docs/expression_language/how_to/message_history/).

![](https://i.imgur.com/8hLJMPl.gif)

### Building on the Q&A RAG System - to a Conversational Q&A RAG System

We will enhance our Q&A RAG System, which utilizes the Wikipedia dataset, by implementing the following updates:

- **Prompt Adjustment:** Our prompt will be modified to include historical messages as inputs, allowing the system to maintain context over the course of a conversation.

- **Contextualizing Questions:** We will introduce a sub-chain mechanism to reformulate the latest user query by considering the chat history. This is crucial for understanding questions that refer back to previous messages. For example, a query like "Can you elaborate on the second point?" relies on the context provided by preceding interactions, which affects the system's ability to retrieve relevant information effectively.





## Contextualizing the Question

To maintain a seamless flow in conversations, especially in a Q&A setting, it's essential to incorporate historical interactions. Here’s how we achieve this:

### Defining a Sub-Chain for Historical Context

1. **Sub-Chain Creation:** We'll define a sub-chain that uses both historical messages and the latest user query. This sub-chain reformulates the question if it refers to any past interactions, ensuring the system, especially the vector database understands the context to return the most relevant documents to this newly reworded question.

2. **Using `MessagesPlaceholder`:** Our prompt construction involves a `MessagesPlaceholder` variable named `chat_history`. This setup allows us to input a list of messages using the `chat_history` key. The system integrates these messages, positioning them after its own responses and before the latest user question.

3. **Helper Function Usage:** We employ the `create_history_aware_retriever` function available [here](https://api.python.langchain.com/en/latest/chains/langchain.chains.history_aware_retriever.create_history_aware_retriever.html). This function is crucial for handling instances where the chat history might be empty and orchestrates the sequence of operations: `prompt | llm | StrOutputParser() | retriever`.

4. **Chain Construction:** The `create_history_aware_retriever` constructs a chain that processes inputs under the keys `input` and `chat_history`, ensuring the output schema aligns with that of a retriever.

By implementing these steps, our system can effectively utilize historical context to better understand and respond to user queries, thereby enhancing the conversational experience.


In [0]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rephrase_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    chatgpt, similarity_retriever, rephrase_prompt
)

This chain prepends a rephrasing of the input query to our retriever, so that the retrieval incorporates the context of the conversation.

## Building the QA RAG Chain with Chat History

Now we're ready to construct our comprehensive QA RAG chain, which leverages historical context for more accurate and relevant responses.

### Components of the QA RAG Chain

1. **Creating Document Chains:**
   - We use the `create_stuff_documents_chain` function, which is detailed [here](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html). This function is used to create a `question_answer_chain`, accepting inputs such as `context`, `chat_history`, and `input`. It efficiently combines the retrieved context with the conversation history and the current query to generate an informed answer.

2. **Building the Final QA RAG Chain:**
   - The entire QA RAG chain is assembled using the `create_retrieval_chain` function, available [here](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html). This chain integrates the `history_aware_retriever` with the `question_answer_chain`. It retains intermediate outputs like the retrieved context for added convenience during the query handling process.
   - The `create_retrieval_chain` function accepts keys such as `input` and `chat_history` and includes `input`, `chat_history`, `context`, and `answer` in its outputs.

By implementing these steps, the system not only contextualizes but also provides accurate answers by synthesizing information from both the current and historical interactions. This method enhances the conversational AI’s ability to understand and respond to user queries dynamically, making the interactions more engaging and relevant.


In [0]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks.
                      Use the following pieces of retrieved context to answer the question.
                      If the answer is not present in the context, just say that you don't know.
                      Keep the answer to the point.

                      Context:
                      {context}

                  """

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", """Question:
                     {input}

                     Answer:
                  """),
    ]
)

question_answer_chain = create_stuff_documents_chain(chatgpt, qa_prompt)
qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [0]:
chat_history = []

question = "What is the capital of India?"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])

In [0]:
chat_history

In [0]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])
chat_history

In [0]:
question = "Tell me more about this city"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])

In [0]:
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])
chat_history

In [0]:
question = "What is the fastest animal?"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
print(response['answer'])

In [0]:
response

In [0]:
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])

In [0]:
chat_history

In [0]:
chat_history[-2:]

In [0]:
question = "Tell me about its different species"
response = qa_rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question),
                     AIMessage(content=response["answer"])])
print(response['answer'])

# Multi-User Conversational RAG System with LangChain

In many Q&A applications, the ability to engage in back-and-forth conversations with users is crucial. This necessitates the application having a form of "memory" to recall past interactions and apply this context to current queries.

However in most real-world conversational systems, multiple users or user sessions will be accessing the system simultaneously.

![](https://i.imgur.com/X4WivLu.gif)

Here we will show how you can use `SQLChatMessageHistory` such that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time. Instead of in-memory we can store it in a SQL database which can be used to store a lot of conversations.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in a SQL database. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

We also use a `memory_buffer_window` function to only use the top-K last historical conversations before sending it to the LLM, basically our own implementation of `ConversationBufferWindowMemory`




In [0]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove all conversation histories
!rm memory.db

In [0]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

######### REPHRASER ############
rephrase_system_prompt = """Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is."""

rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rephrase_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    chatgpt, similarity_retriever, rephrase_prompt
)



######### MULTI_USER RAG RESPONSE GENERATOR ############
qa_system_prompt = """You are an assistant for question-answering tasks.
                      Analyze the user question carefully and use the following pieces of
                      retrieved context to answer the question.
                      If the answer is not present in the context, just say that you don't know.
                      Keep the answer to the point.

                      Context:
                      {context}
                  """

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", """Question:
                     {input}

                     Answer:
                  """),
    ]
)

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    history = SQLChatMessageHistory(session_id, "sqlite:///memory.db")
    return history

# subset historical conversations based on last K conversation messages
# here by default we use the last 10 conversations (ai-human) as memory to the input prompt
def memory_buffer_window(messages, lastk_conversations=10):
    return messages[-(lastk_conversations*2):] # each conversation has 2 messages - (human prompt, AI response)

# custom RAG chain which looks at last K conversational messages
question_answer_chain = (
    RunnablePassthrough.assign(chat_history=lambda x: memory_buffer_window(x["chat_history"]))
      |
    qa_prompt
      |
    chatgpt
      |
    StrOutputParser()
)
qa_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


############ CONVERSATIONAL RAG CHAIN ####################
conversational_rag_chain = RunnableWithMessageHistory(
    qa_rag_chain,
    get_session_history_db,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [0]:
from IPython.display import display, Markdown

def conv_rag_chatbot(usersession_id, prompt):
    response = conversational_rag_chain.invoke(
                                {"input": prompt},
                                config={
                                    "configurable": {"session_id": usersession_id}
                                }
    )
    print('Answer:')
    display(Markdown(response['answer']))
    print('Sources:')
    for document in response['context']:
        print(document)
        print()

    return response

In [0]:
us_id = 'bond007'
r = conv_rag_chatbot(us_id, 'What is the capital of India?')

In [0]:
r = conv_rag_chatbot(us_id, 'Tell me more about it')

In [0]:
us_id = 'jim003'
r = conv_rag_chatbot(us_id, 'What is the fastest animal on land?')

In [0]:
us_id = 'bond007'
r = conv_rag_chatbot(us_id, 'Tell me about wildlife in India')

In [0]:
us_id = 'jim003'
r = conv_rag_chatbot(us_id, 'tell me more about its different species')